In [1]:
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
import os
from io import StringIO
import boto3
import json

load_dotenv() 

host = os.getenv("HOST")
port = os.getenv("PORT")
database = os.getenv("DATABASE")
user = os.getenv("USER")
password = os.getenv("PASSWORD")
bucket = os.getenv('BUCKET')



**GET THE CSV FROM S3 BUCKET**

In [2]:
file_name = 'youtube-video-stats.csv'
s3_client =boto3.client('s3')
try:
    response = s3_client.get_object(Bucket = bucket, Key = file_name)
    print('Get object - Success')
except Exception as e:
    print(f"An error occurred: {e}")

Get object - Success


In [3]:
csv_content = response['Body'].read().decode('utf-8')
df_staging = pd.read_csv(StringIO(csv_content))

**SIMPLY PRUNE SOME COLUMNS**

In [4]:
df_staging = df_staging[['Title', 'Video ID', 'Published At', 'Keyword', 'Likes', 'Comments', 'Views']]
df_staging['Published At'] = pd.to_datetime(df_staging['Published At'])

In [5]:
df_worked = df_staging
df_worked

,Title,Video ID,Published At,Keyword,Likes,Comments,Views
0,Apple Pay Is Killing the Physical Wallet After...,wAZZ-UWGVHI,2022-08-23,tech,3407.0,672.0,135612.0
1,The most EXPENSIVE thing I own.,b3x28s61q3c,2022-08-24,tech,76779.0,4306.0,1758063.0
2,My New House Gaming Setup is SICK!,4mgePWWCAmA,2022-08-23,tech,63825.0,3338.0,1564007.0
3,Petrol Vs Liquid Nitrogen | Freezing Experimen...,kXiYSI7H2b0,2022-08-23,tech,71566.0,1426.0,922918.0
4,Best Back to School Tech 2022!,ErMwWXQxHp0,2022-08-08,tech,96513.0,5155.0,1855644.0
...,...,...,...,...,...,...,...
1876,Should You Learn Machine Learning?,AO6urf07KjE,2021-06-14,machine learning,10259.0,416.0,386360.0
1877,Todos podemos aprender Machine learning,7ClLKBUvmRk,2017-10-08,machine learning,2981.0,72.0,431421.0
1878,"Andrew Ng: Deep Learning, Education, and Real-...",0jspaMLxBig,2020-02-20,machine learning,5198.0,443.0,226152.0
1879,What is Machine Learning?,f_uwKZIAeM0,2017-01-11,machine learning,NaN,NaN,NaN


**CREATE CONNECTION TO THE DATABASE**

In [6]:
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")


**INSERT THE DATAFRAME INTO A RDS DATABASE**

In [7]:
try:
    df_worked.to_sql('youtube_stats', con = engine, if_exists = 'replace')
    print('Table created succesfully')
except Exception as e:
    print(f"An error occurred: {e}")

Table created succesfully
